In [3]:
!pip install stellargraph[demos]

  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)


  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.0
    Uninstalling numpy-1.18.0:


      Successfully uninstalled numpy-1.18.0
  Rolling back uninstall of numpy
  Moving to /home/jovyan/.local/bin/f2py
   from /tmp/pip-uninstall-85uc92cz/f2py
  Moving to /home/jovyan/.local/bin/f2py3
   from /tmp/pip-uninstall-85uc92cz/f2py3
  Moving to /home/jovyan/.local/bin/f2py3.7
   from /tmp/pip-uninstall-85uc92cz/f2py3.7
  Moving to /home/jovyan/.local/lib/python3.7/site-packages/numpy-1.18.0.dist-info/
   from /home/jovyan/.local/lib/python3.7/site-packages/~umpy-1.18.0.dist-info
  Moving to /home/jovyan/.local/lib/python3.7/site-packages/numpy/
   from /home/jovyan/.local/lib/python3.7/site-packages/~-mpy
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/home/user/conda/lib/python3.7/site-packages/numpy/_distributor_init.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the '/home/user/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install networkx

  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
You should consider upgrading via the '/home/user/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 500)
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

from stellargraph import globalvar

from stellargraph import datasets
from IPython.display import display, HTML

In [6]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML

## Подготовка датасета

Загрузка и предобработка метаданных о научных статьях:

In [25]:
meta = pd.read_csv("citatio.csv", index_col = 0)

In [26]:
meta["abs_emb"] = meta["abs_emb"].map(lambda x: x.replace("\n", "").replace("[", "").replace("]", "").split(" "))

In [27]:
meta["abs_emb"] = meta["abs_emb"].map(lambda x: [y for y in x if y])

In [28]:
node_data = meta["abs_emb"]
node_data = pd.DataFrame(list(node_data))

Датафрейм с векторизованными аннотациями (fasttext) статей:

In [29]:
node_data = node_data.join(meta["paper_id"]).set_index("paper_id")
node_data

,0,1,2,3,4,5,6,7,8,9
paper_id,,,,,,,,,,
1001,-0.72798234,0.47327074,-0.09284023,0.6732989,-0.23567905,-0.6464117,-0.18148245,-0.43541744,0.23336542,-0.9543899
1002,-0.8018193,0.24585508,-0.10499282,0.84289485,-0.37486026,-0.4185827,0.05874766,-0.05334255,0.73340255,-0.4718775
1003,-0.60231924,0.2232491,-0.18635671,0.86939037,-0.22906514,-0.22417377,0.10357788,-0.20258087,0.6444224,-0.7061434
1004,-0.21616772,0.4057547,-0.05586757,0.7138989,-0.25783542,-0.3352258,0.2834404,-0.15918884,0.6370065,-0.8274329
1005,-0.27283168,0.39040273,-0.27094933,0.6896722,-0.12617603,-0.49113902,-0.17617069,-0.5251945,0.44413218,-0.8203365
...,...,...,...,...,...,...,...,...,...,...
9912290,-0.04657721,0.41355953,-0.21967253,0.7020267,-0.00193853,-0.37004966,-0.2063545,-0.1278213,0.4918682,-1.1466386
9912291,-0.08927432,0.39376086,-0.060005,0.85654616,-0.2299481,-0.06304786,0.2707951,0.05544405,0.68550444,-0.8276629
9912292,-0.46030122,0.26681346,-0.3795974,0.87863266,0.00683328,-0.27136758,-0.17363684,-0.40357453,0.634274,-0.58183014


Загрузка структурного представления графа цитирования в виде списка ребер:

In [30]:
edges = pd.read_csv("hep-th-citations.csv", header = None, sep = " ")

In [31]:
edge_data = pd.DataFrame(
    {
        "source": list(edges[0]),
        "target": list(edges[1])
    }
)
edge_data

,source,target
0,1001,9304045
1,1001,9308122
2,1001,9309097
3,1001,9311042
4,1001,9401139
...,...,...
352802,9912293,9907211
352803,9912293,9908105
352804,9912293,9908142
352805,9912293,9909081


Преобразуем структурное и признаковое описание в единый граф:

In [32]:
G = sg.StellarGraph(
    {"street": node_data}, {"connection": edge_data}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 29555, Edges: 352807

 Node types:
  street: [29555]
    Features: float32 vector, length 10
    Edge types: street-connection->street

 Edge types:
    street-connection->street: [352807]
        Weights: all 1 (default)
        Features: none


## Построение эмбеддингов статей

Задаем параметры GraphSAGE:

In [33]:
nodes = list(G.nodes())
number_of_walks = 2
length = 5

In [40]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)

In [41]:
batch_size = 50
epochs = 5
num_samples = [10, 5]

In [42]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [43]:
layer_sizes = [30, 30]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2"
)

In [37]:
x_inp, x_out = graphsage.in_out_tensors()

Выбираем метод построения эмбеддингов ребер:

In [38]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [24]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(run_eagerly=True,
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [ ]:
history = model.fit(
    train_gen,
    epochs=15,
    verbose=1,
    use_multiprocessing=True,
    workers=22,
    shuffle=True,
)

Epoch 1/15
8887/8887 [==============================] - 665s 74ms/step - loss: 0.5772 - binary_accuracy: 0.7175
Epoch 2/15
8887/8887 [==============================] - 694s 78ms/step - loss: 0.5641 - binary_accuracy: 0.7343
Epoch 3/15
8887/8887 [==============================] - 670s 75ms/step - loss: 0.5634 - binary_accuracy: 0.7362
Epoch 4/15
8887/8887 [==============================] - 687s 77ms/step - loss: 0.5625 - binary_accuracy: 0.7369
Epoch 5/15
8887/8887 [==============================] - 701s 78ms/step - loss: 0.5620 - binary_accuracy: 0.7392
Epoch 6/15
8887/8887 [==============================] - 674s 75ms/step - loss: 0.5618 - binary_accuracy: 0.7408
Epoch 8/15
8887/8887 [==============================] - 693s 77ms/step - loss: 0.5617 - binary_accuracy: 0.7406
Epoch 9/15
8887/8887 [==============================] - 694s 78ms/step - loss: 0.5613 - binary_accuracy: 0.7428
Epoch 10/15
8887/8887 [==============================] - 736s 82ms/step - loss: 0.5607 - binary_accuracy

Получаем эмбеддинги вершин (статей):

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from stellargraph.mapper import GraphSAGENodeGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_ids = G.nodes()
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

In [57]:
pd.DataFrame(node_embeddings).to_csv("hepth_cit_emb.csv")

NameError: name 'node_embeddings' is not defined

In [56]:
paper_embeddings = pd.DataFrame(node_embeddings).join(pd.DataFrame(list(G.nodes()), columns = ["true_index"])).set_index("true_index")

NameError: name 'node_embeddings' is not defined

In [99]:
paper_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
true_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,-0.062178,-0.094036,-0.185020,-0.047650,-0.096164,0.172644,0.134685,0.198581,0.093734,0.088871,-0.038374,-0.055188,-0.051739,-0.233399,0.072453,0.196949,0.323212,0.014371,-0.138927,-0.110285,0.042071,0.406208,0.438242,0.016824,-0.146370,-0.218347,0.090305,-0.068023,-0.031999,0.391935
1002,0.054775,-0.032364,0.070146,0.119664,0.050044,0.043663,-0.101795,-0.004838,0.037778,0.047285,-0.034745,-0.299143,-0.015524,-0.102862,0.083557,0.360558,0.223833,0.039485,0.000408,-0.381716,-0.360845,0.315523,0.306413,0.180319,0.034374,0.015151,-0.104933,0.256311,-0.279516,-0.099862
1003,0.034642,0.027232,0.150959,0.100759,0.100580,-0.073592,-0.146519,-0.020694,0.011322,0.000116,-0.054486,-0.190480,0.001633,0.009268,0.014148,0.229470,0.009994,0.046259,-0.019798,-0.433784,-0.431271,0.162568,0.050904,0.172220,0.164044,0.163469,-0.107573,0.335009,-0.338688,-0.328245
1004,0.006335,-0.011994,0.225443,0.076603,0.033043,-0.163899,-0.091215,-0.037141,-0.042736,-0.026595,0.138537,0.009031,0.034771,0.104728,0.078402,0.162550,-0.034892,0.144266,-0.020576,0.003058,-0.198470,-0.472837,-0.173457,-0.051672,0.328445,0.028987,-0.163827,0.272799,0.004289,-0.557437
1005,-0.027137,-0.064572,-0.200007,-0.173864,-0.132871,0.234220,0.154205,0.088636,-0.023434,-0.031053,-0.119528,0.017047,0.014193,-0.010157,0.045359,-0.127152,0.030748,-0.085875,0.054661,0.303920,0.306768,0.101420,0.033873,0.048732,-0.339202,-0.192329,0.217454,-0.313982,0.198767,0.486904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912290,-0.039358,-0.020489,-0.074166,-0.086550,-0.076258,0.052410,0.189791,0.082669,-0.016660,-0.002829,0.054623,0.240443,0.012031,0.092855,0.040759,-0.164136,-0.120036,0.126387,0.316895,0.511174,0.199937,-0.481008,-0.148334,0.095693,-0.014092,0.039092,-0.143638,0.117057,0.319586,-0.048680
9912291,0.011311,-0.015671,0.190527,0.051188,-0.009977,-0.184064,-0.058968,-0.052833,-0.028627,-0.048613,0.141619,0.039910,0.033253,0.080870,0.077178,0.065388,-0.048607,0.107871,0.022387,0.089930,-0.143245,-0.541278,-0.280069,-0.109149,0.313632,0.075075,-0.135097,0.187871,0.117546,-0.529097
9912292,0.063488,0.061054,0.028156,0.004342,0.127167,0.110052,-0.167547,-0.066928,-0.055451,-0.064535,-0.258218,-0.319575,-0.011777,0.055325,-0.054500,-0.240723,-0.323840,-0.158075,-0.004731,-0.246057,-0.088794,0.406004,-0.134330,0.191807,-0.254403,0.100837,0.180359,-0.132125,-0.317837,0.215765


## Предобрабокта данных графа соавторства

In [44]:
authors_edges = pd.read_csv("extended_edge_list.csv", index_col = 0)

In [45]:
authors_nodes = pd.read_csv("author_feature_list.csv", index_col = "author_id").drop(["Unnamed: 0"], axis = 1)

In [46]:
authors_nodes["interests"] = authors_nodes["interests"].map(lambda x: x.replace("{", "").replace("}", "").split(","))
authors_nodes

,interests
author_id,
0,"[0, 33, 8, 44, 47]"
1,"[0, 33, 45, 8]"
2,"[0, 1, 33, 3, 8, 44, 45, 47]"
3,"[8, 24, 34]"
4,"[0, 8, 26, 47]"
...,...
12920,"[8, 33]"
12921,"[0, 33, 8, 43, 44, 45, 47, 24]"
12922,"[0, 1, 33, 8, 12, 44, 14, 45]"


In [47]:
for i in range(55):
    authors_nodes[int(i)] = 0 

In [48]:
for i in tqdm(range(len(authors_nodes))):
#    print("new")
    for j in range(len(authors_nodes["interests"][i])):
#        print(int(authors_nodes["interests"][i][j]))
        authors_nodes[int(authors_nodes["interests"][i][j])][i] = 1 
    

  0%|          | 0/12925 [00:00<?, ?it/s]/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 12925/12925 [00:04<00:00, 2989.84it/s]


In [49]:
authors_nodes = authors_nodes.drop(["interests"], axis = 1)


In [50]:
authors_edges["article_ids"] = authors_edges["article_ids"].map(lambda x: x.replace("{", "").replace("}", "").split(","))

In [51]:
authors_edges

,from_id,to_id,article_ids
0,8950,7794,"[9705187, 9808012, 9803118, 9708145, 98100..."
1,9308,3382,[9509068]
2,9308,3432,[9509068]
3,3382,3432,[9509068]
4,9105,673,"[211203, 212139, 207127]"
...,...,...,...
24016,9922,9388,[6021]
24017,2700,3331,[9708101]
24018,755,12203,[9406137]
24019,755,4805,[9406137]


In [54]:
def summ(first, second):
    return [x + y for x, y in zip(first, second)]

Создаем признаковое описание вершин на основе данных об интересах ученых со scholars и сумме эмбеддингов ассоциированных с ними статей:

In [55]:
from tqdm import tqdm 
collab_embedding = []
for i in tqdm(range(len(authors_edges))):
    indexes = authors_edges.loc[i , :]["article_ids"]
    embedding = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for j in range(len(indexes)):
        embedding = summ(embedding, paper_embeddings.loc[int(authors_edges.loc[i , :]["article_ids"][j]), :])
    collab_embedding.append(embedding)

  0%|          | 0/24021 [00:00<?, ?it/s]


NameError: name 'paper_embeddings' is not defined

Признаковое описание ассоциированных статей с вершинами (авторами) в графе соавторства:

In [58]:
collab_embedding

""


Преобразуем структурное и признаковое описание в единый граф:


In [246]:
edge_data_a = pd.DataFrame(
    {
        "source": list(authors_edges["from_id"]),
        "target": list(authors_edges["to_id"])
    }
)


G = sg.StellarGraph(
    {"street": authors_nodes}, {"connection": edge_data_a}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 12925, Edges: 24021

 Node types:
  street: [12925]
    Features: float32 vector, length 55
    Edge types: street-connection->street

 Edge types:
    street-connection->street: [24021]
        Weights: all 1 (default)
        Features: none


##  Обучение GraphSAGE без данных о работах авторов

In [247]:
nodes = list(G.nodes())
number_of_walks = 1
length = 5
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)
batch_size = 50
epochs = 5
num_samples = [10, 5]
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)
layer_sizes = [30, 30]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2"
)
x_inp, x_out = graphsage.in_out_tensors()
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(run_eagerly=True,
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)
history = model.fit(
    train_gen,
    epochs=5,
    verbose=1,
    use_multiprocessing=True,
    workers=22,
    shuffle=True,
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/5
1863/1863 [==============================] - 61s 32ms/step - loss: 0.5864 - binary_accuracy: 0.6993
Epoch 2/5
1863/1863 [==============================] - 60s 32ms/step - loss: 0.5560 - binary_accuracy: 0.7381
Epoch 3/5
1863/1863 [==============================] - 61s 32ms/step - loss: 0.5567 - binary_accuracy: 0.7322
Epoch 4/5
1863/1863 [==============================] - 61s 32ms/step - loss: 0.5527 - binary_accuracy: 0.7398
Epoch 5/5
1863/1863 [==============================] - 60s 32ms/step - loss: 0.5523 - binary_accuracy: 0.7387


Признаковое описание авторов (интересы + статьи):

In [253]:
authors_nodes_e = authors_nodes.copy()

In [300]:
for i in tqdm(range(len(authors_edges))):
    id_from = authors_edges["from_id"][i]
    id_to = authors_edges["to_id"][i]
    emb = collab_embedding.loc[i]
    for j in range(30):
        authors_nodes_e.loc[id_from]["e" + str(j)] += emb[j]
    for j in range(30):
        authors_nodes_e.loc[id_to]["e" + str(j)] += emb[j]


 20%|██        | 4879/24021 [00:29<01:54, 167.18it/s]


 41%|████      | 9740/24021 [00:58<01:25, 166.81it/s]


 61%|██████    | 14534/24021 [01:27<00:57, 165.36it/s]


 80%|████████  | 19328/24021 [01:55<00:28, 166.53it/s]


100%|██████████| 24021/24021 [02:24<00:00, 166.60it/s]


In [301]:
 authors_nodes_e

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29
author_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.761014,0.426571,-0.034670,0.246092,1.100836,1.103972,-0.666413,-0.216230,-0.287217,0.061704,-1.545421,-1.686130,-0.097440,0.820776,-0.354818,-1.385652,-2.467687,-0.345646,2.171595,-0.609744,-0.318269,2.843063,0.335700,3.126533,-2.683764,1.063220,-0.115769,0.472406,-1.762113,2.031837
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309202,-0.110332,-0.031818,0.069162,-0.189481,0.001254,0.662915,-0.661328,-0.503896,-0.349884,0.509949,1.218944,0.133060,0.822702,0.269428,0.371611,1.099969,0.120785,1.651843,1.883549,0.373510,-1.613560,-0.180801,0.022839,0.262162,0.754593,-0.980078,0.550949,2.095026,-0.363222
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.226610,0.134537,0.082813,0.113141,-0.052590,-0.247539,0.519333,1.239638,0.393311,0.951188,0.465181,2.049801,0.041263,0.677290,-0.537604,-0.056283,-1.141393,2.550951,-0.633066,0.137032,1.310421,-1.347399,1.255367,-0.053356,1.390591,-0.342350,-1.239332,1.526059,0.863807,-0.365091
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12921,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159481,-0.353896,0.763497,0.822761,-0.070898,-0.386178,-0.701763,-0.00097

In [304]:
authors_nodes_e.to_csv("AAA_node_features_extended.csv")

In [305]:
authors_nodes.to_csv("AAA_node_features.csv")

In [306]:
authors_edges.to_csv("AAA_edges.csv")

Преобразуем структурное и признаковое описание в единый граф:

In [302]:
edge_data_a = pd.DataFrame(
    {
        "source": list(authors_edges["from_id"]),
        "target": list(authors_edges["to_id"])
    }
)


G = sg.StellarGraph(
    {"street":  authors_nodes_e}, {"connection": edge_data_a}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 12925, Edges: 24021

 Node types:
  street: [12925]
    Features: float32 vector, length 85
    Edge types: street-connection->street

 Edge types:
    street-connection->street: [24021]
        Weights: all 1 (default)
        Features: none


##  Обучение GraphSAGE с данными о работах авторов

In [303]:
nodes = list(G.nodes())
number_of_walks = 1
length = 5
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)
batch_size = 50
epochs = 5
num_samples = [10, 5]
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)
layer_sizes = [30, 30]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2"
)
x_inp, x_out = graphsage.in_out_tensors()
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(run_eagerly=True,
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)
history = model.fit(
    train_gen,
    epochs=10,
    verbose=1,
    use_multiprocessing=True,
    workers=22,
    shuffle=True,
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/5
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5598 - binary_accuracy: 0.7399
Epoch 2/5
1863/1863 [==============================] - 63s 34ms/step - loss: 0.5434 - binary_accuracy: 0.7596
Epoch 3/5
1863/1863 [==============================] - 63s 34ms/step - loss: 0.5410 - binary_accuracy: 0.7579
Epoch 4/5
1863/1863 [==============================] - 67s 35ms/step - loss: 0.5391 - binary_accuracy: 0.7637
Epoch 5/5
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5380 - binary_accuracy: 0.7663


In [307]:
history = model.fit(
    train_gen,
    epochs=10,
    verbose=1,
    use_multiprocessing=True,
    workers=22,
    shuffle=True,
)

Epoch 1/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5385 - binary_accuracy: 0.7662
Epoch 2/10
1863/1863 [==============================] - 63s 34ms/step - loss: 0.5379 - binary_accuracy: 0.7651
Epoch 3/10
1863/1863 [==============================] - 63s 33ms/step - loss: 0.5384 - binary_accuracy: 0.7647
Epoch 4/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5375 - binary_accuracy: 0.7666
Epoch 5/10
1863/1863 [==============================] - 63s 34ms/step - loss: 0.5381 - binary_accuracy: 0.7641
Epoch 6/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5378 - binary_accuracy: 0.7664
Epoch 7/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5378 - binary_accuracy: 0.7675
Epoch 8/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5378 - binary_accuracy: 0.7667
Epoch 9/10
1863/1863 [==============================] - 64s 34ms/step - loss: 0.5370 - binary_accuracy: 0.7667
E

Модель #2:

In [142]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [143]:
modelNN = tf.keras.models.Sequential()
modelNN.add(tf.keras.Input(shape=(72,)))
modelNN.add(tf.keras.layers.Dense(128, activation='relu'))
modelNN.add(tf.keras.layers.Dense(128*2, activation='relu'))
modelNN.add(tf.keras.layers.Dense(128*4, activation='relu'))
modelNN.add(tf.keras.layers.Dense(1))

In [144]:
modelNN.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [145]:
train_x = np.asarray(X.iloc[0:700000, 0:72])
train_y = np.asarray(Y[0:700000])

In [146]:
test_x = np.asarray(X.iloc[700000:740000, 0:72])
test_y = np.asarray(Y[700000:740000])

In [147]:
from  tensorflow.keras.utils import normalize, to_categorical
train_x = normalize(train_x, axis=1)
test_x =  normalize(test_x, axis=1)

In [151]:
historyNN = modelNN.fit(train_x, train_y, epochs = 10)

Epoch 1/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6723
Epoch 2/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6416
Epoch 3/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6533
Epoch 4/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6568
Epoch 5/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6234
Epoch 6/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.6289
Epoch 7/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.5763
Epoch 8/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.5663
Epoch 9/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.5817
Epoch 10/10
21875/21875 [==============================] - 30s 1ms/step - loss: 389.5744


In [152]:
result = modelNN.evaluate(
    test_x,
    test_y, verbose=0)

In [153]:
result

388.2742919921875